In [1]:
import PySimpleGUI as sg
from pathlib import Path
import pandas as pd

import os 
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [5]:
def pred_pop(window, x, y, z, a):
    
    hours = []
    
    for i in range(6, 24):
        hours.append(i*100)
        hours.append(i*100+15)
        hours.append(i*100+30)
        hours.append(i*100+45)
    
    layout = [
        [sg.Text("Please choose a time (hhmm)")],
        [sg.Listbox(hours, size=(20,4), enable_events=False, key='-LIST-')],
        [sg.Button("Ok"), sg.Button("Cancel")]
    ]
    
    window = sg.Window("Predictions", layout, modal=True, grab_anywhere=True, enable_close_attempted_event=True)
    event, value = window.read()
    
    if event == sg.WINDOW_CLOSE_ATTEMPTED_EVENT:
        event = "CANCEL"
    
    if event == "Ok":
        time = map(int, value['-LIST-'])
        time = str(time)
        people = 13
        temp = 5
        hum = 20
        sg.popup("At the time selected,\n"+str(people)+" people are expected, \nwith a temperature of "+str(temp)+"C, and humidity of "+str(hum)+"%")
        
    
    window.close()


In [6]:
def graph_pop(window, x, y, z, a):
    
    #def update_figure(x, y):
    #    axes = fig.axes
    #    plot(x, y, 'r-')
    #    figure_canvas_agg.draw()
    #    figure_canvas_agg.get_tk_widget().pack()
    
    
    data = []
    for i in range(0, len(x)):
        data.append([x[i], y[i], z[i], a[i]])
    
    layout = [
        [sg.Text("Recordings depending on time")],
        #[sg.Table(headings = ["Time", "Observations"], 
        #         values = data,
        #         expand_x = True,
        #         hide_vertical_scroll = True,
        #         key = '-TABLE-')],
        [sg.Canvas(key = '-CANVAS-')],
        [sg.Button("Exit")]
    ]
    
    window = sg.Window("Graph", layout, modal=True, grab_anywhere=True, enable_close_attempted_event=True, finalize=True)
    fig = matplotlib.figure.Figure(figsize = (15, 5))
    
    fig.add_subplot(131, xlabel = 'Time Recorded', ylabel = 'People', title = 'Flow of people').plot(x, y)
    
    fig.add_subplot(132, xlabel = 'Time Recorded', ylabel = 'Temperature (C)', title = 'Flow of Temperature').plot(x, z)
    
    fig.add_subplot(133, xlabel = 'Time Recorded', ylabel = 'Humidity (%)', title = 'Flow of Humidity').plot(x, a)
    
    figure_canvas_agg = FigureCanvasTkAgg(fig,window['-CANVAS-'].TKCanvas)
    figure_canvas_agg.draw()
    figure_canvas_agg.get_tk_widget().pack()
    
  
    event, values = window.read()
      
    if event == sg.WINDOW_CLOSE_ATTEMPTED_EVENT:
        event = "CANCEL"
        
    window.close()

In [7]:
sg.theme("BlueMono")
sg.set_options(font=("Courier New", 16))

menu_layout = [
    ['File', ['Open', '---', 'Exit']],
    ['Tools', ['Predictions', 'Graphs']]
]

data = []
update = []
layout = [  
    [sg.Menu(menu_layout)],
    [sg.Text('LIBMON', key = '-DOCNAME-')],
    # [sg.Multiline(no_scrollbar = True, size = (30, 10), key = '-TEXTBOX-')],
    [sg.Table(headings = ["Time Rec", "People", "Temperature", "Humidity"], 
                 values = data,
                 expand_x = True,
                 hide_vertical_scroll = True,
                 key = '-TABLE-')]
     ]

# Create the Window
window = sg.Window('FOREsight', layout)

# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Exit': # if user closes window or clicks cancel
        break
        
    if event == 'Open':
        file_path = sg.popup_get_file('Insert LIBMON file', no_window=True, default_extension='txt')
        if file_path:
            file = Path(file_path)
            data = np.loadtxt(file, delimiter=",", skiprows=1)
            window['-DOCNAME-'].update(file_path.split('/')[-1])
            x = data[:,0]
            y = data[:,1]
            z = data[:,2]
            a = data[:,3]
            for i in range(0, len(x)):
                update.append([x[i], y[i], z[i], a[i]])
            window['-TABLE-'].update(update)
            
    
    if event == 'Graphs':
        # Display the table in a window
        graph_pop(window, x, y, z, a)
        
    if event == 'Predictions':
        #preds = np.loadtxt(file, delimiter=",", skiprows=1)
        #x = preds[:,0]
        #y = preds[:,1]
        #z = preds[:,2]
        #a = preds[:,3]
        pred_pop(window, x, y, z, a)
    
window.close()